In [2]:
!pip install pyjwt

  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
Using cached PyJWT-2.10.1-py3-none-any.whl (22 kB)


In [5]:
import jwt
import requests

In [2]:
import json
import time
import jwt  # PyJWT library (install with `pip install PyJWT`)
import requests

# Path to your Google Cloud JSON key
KEY_PATH = "google_api.json"

# Load JSON credentials
with open(KEY_PATH) as f:
    creds = json.load(f)

# Google OAuth 2.0 Token URL
GOOGLE_OAUTH2_TOKEN_URL = "https://oauth2.googleapis.com/token"

def get_access_token():
    """Generate an OAuth 2.0 access token manually using service account JSON."""
    
    # Define JWT claim set
    issued_at = int(time.time())
    expiration_time = issued_at + 3600  # Token valid for 1 hour

    payload = {
        "iss": creds["client_email"],  # Issuer (Service Account Email)
        "scope": "https://www.googleapis.com/auth/cloud-translation",  # Translation API scope
        "aud": GOOGLE_OAUTH2_TOKEN_URL,
        "iat": issued_at,  # Issued At
        "exp": expiration_time  # Expiry
    }

    # Create a JWT signed with the private key
    private_key = creds["private_key"]
    jwt_token = jwt.encode(payload, private_key, algorithm="RS256")

    # Exchange JWT for OAuth 2.0 access token
    token_request_data = {
        "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
        "assertion": jwt_token
    }

    response = requests.post(GOOGLE_OAUTH2_TOKEN_URL, data=token_request_data)
    
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Error obtaining access token: {response.text}")

def translate(text, target_lang="es"):
    """Use Google Cloud Translation API with manually obtained access token."""
    
    token = get_access_token()
    url = "https://translation.googleapis.com/language/translate/v2"

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    payload = {
        "q": text,
        "target": target_lang
    }

    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 200:
        return response.json()["data"]["translations"][0]["translatedText"]
    else:
        return f"Error: {response.text}"

# Example Usage
translated_text = translate("Hello, how are you?", target_lang="pl")
print(translated_text)  # Output: Bonjour, comment ça va ?


Witam, jak się masz?


In [3]:
!pip install pyjwt

In [6]:
import requests
import json
import jwt  # PyJWT library
import time

def google_translate(text, target_language="pl", project_id="magisterka-454811", credentials_path="google_api.json"):
    """
    Translates text using Google Translate API v3 with manual JWT authentication.

    Args:
        text (str): The text to translate.
        target_language (str): The target language code (default is "pl" for Polish).
        project_id (str): Your Google Cloud project ID.
        credentials_path (str): Path to the service account JSON file.

    Returns:
        str: The translated text.
    """
    # Define the API endpoint
    url = f"https://translation.googleapis.com/v3/projects/{project_id}/locations/global:translateText"

    # Load the service account JSON file
    with open(credentials_path, "r") as file:
        service_account_info = json.load(file)

    # Create a JWT for authentication
    now = int(time.time())
    payload = {
        "iss": service_account_info["client_email"],
        "sub": service_account_info["client_email"],
        "aud": "https://oauth2.googleapis.com/token",  # Correct audience for token exchange
        "iat": now,
        "exp": now + 3600,  # Token valid for 1 hour
    }
    private_key = service_account_info["private_key"]
    jwt_token = jwt.encode(payload, private_key, algorithm="RS256")

    # Exchange the JWT for an access token
    token_url = "https://oauth2.googleapis.com/token"
    token_payload = {
        "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
        "assertion": jwt_token,
    }
    token_response = requests.post(token_url, data=token_payload)
    if token_response.status_code != 200:
        raise Exception(f"Error obtaining access token: {token_response.status_code}, {token_response.text}")
    access_token = token_response.json()["access_token"]

    # Prepare the request payload for translation
    payload = {
        "contents": [text],
        "targetLanguageCode": target_language,
        "mimeType": "text/plain",  # Specify the content type
    }

    # Add the access token to the request headers
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
    }

    # Send the request to the Translation API
    response = requests.post(url, json=payload, headers=headers)

    # Check for errors
    if response.status_code != 200:
        raise Exception(f"Error with Google Translate API: {response.status_code}, {response.text}")

    # Parse the response
    result = response.json()
    return result["translations"][0]["translatedText"]

# Example usage
translated_text = google_translate(
    "Hello, how are you?",
    target_language="pl",
    project_id="magisterka-454811",
    credentials_path="google_api.json"
)
print(translated_text)

Exception: Error obtaining access token: 400, {"error":"invalid_scope","error_description":"Invalid OAuth scope or ID token audience provided."}